In [1]:
import os
import napari
import numpy as np
import pandas as pd
import re
import math
from scipy import stats
import skimage as ski
from bioio import BioImage
import bioio_tifffile

In [2]:
# Minimum number of pixels for a filament to be included in analysis
min_pixels = 3

# Minimum length (um) for a filament to be included in analysis.
# Can select 2D, 3D, or both.
min_length_2D = 0
min_length_3D = 0

In [3]:
def parse_coordinates(coordinate_string, tuple_format=False):
	coords_str = coordinate_string[1:-1]
	coord_list = []
	for match in re.findall(r"\(.*?\)", coords_str):
		coordinate = match[1:-1]
		coord_list.append([float(coord) for coord in coordinate.split(",")])
	if tuple_format:
		return [tuple(coord) for coord in coord_list]
	return np.array(coord_list)

def parse_list(list_string):
	list_str = list_string[1:-1]
	return [int(item) for item in list_str.split(",")]

def get_distance(point_array, dims=3):
	if dims == 2:
		point_array = point_array[:, :2]
	diffs = np.diff(point_array, axis=0)
	distances = np.linalg.norm(diffs, axis=1)
	return np.sum(distances)

def get_radians_2D(p1, p2):
	radians = math.atan2(p2[1] - p1[1], p2[0] - p1[0])
	absolute_radians = np.mod(radians, np.pi)
	return absolute_radians


def get_average_radians(point_array, spacer=5):
	if spacer >= len(point_array):
		spacer = len(point_array) - 1
	xdiff = point_array[spacer:, 0] - point_array[:-spacer, 0]
	ydiff = point_array[spacer:, 1] - point_array[:-spacer, 1]
	radians = np.arctan2(ydiff, xdiff)
	mean_radians = stats.circmean(radians)
	absolute_radians = np.mod(mean_radians, np.pi)
	return absolute_radians

def compareAngles(a, b):
	delta = ((b - a + 180.0) % 360.0) - 180.0
	return np.abs(delta)

def get_branch_angles(df, vertices_col, angle_col, new_col):
	# Keep original order/id
	tmp = df.reset_index().rename(columns={"index": "_idx"})

	# 1) One vertex per row
	ex = tmp[["_idx", angle_col, vertices_col]].explode(vertices_col, ignore_index=True)

	# 2) Self-join on vertex to find neighbors sharing any vertex
	pairs = ex.merge(ex, on=vertices_col, how="inner", suffixes=("_src", "_tgt"))

	# 3) Drop self-pairs and (optionally) duplicate pairs that arise if two rows share multiple vertices
	pairs = pairs[pairs["_idx_src"] != pairs["_idx_tgt"]]
	pairs = pairs.drop_duplicates(["_idx_src", "_idx_tgt"])

	# 4) Vectorized angle diffs from src->tgt
	diffs = compareAngles(pairs[f"{angle_col}_src"].to_numpy(),
						   pairs[f"{angle_col}_tgt"].to_numpy())
	pairs = pairs.assign(diff=diffs)

	# 5) Keep strictly positive diffs, aggregate per source row as list
	pairs = pairs[pairs["diff"] > 0]
	agg = pairs.groupby("_idx_src")["diff"].apply(list).rename(new_col)

	# 6) Join back to original frame
	out = tmp.join(agg, on="_idx").drop(columns=["_idx"])

	# Preserve original index/columns order
	out.index = df.index
	return out

def get_residuals(point_array):
	NDModel = ski.measure.LineModelND()
	JustStartAndEnd = np.stack((point_array[0, :], point_array[-1, :]))
	NDModel.estimate(JustStartAndEnd)
	return np.mean(NDModel.residuals(point_array))

def get_curvature(coords, spacing=1):
	n = coords.shape[0]
	if n < (3 * spacing):
		return np.nan, np.nan
	xy = coords[:, :2]

	curv = []
	for i in range(spacing, n - spacing):
		e1, p, e2 = xy[i - spacing], xy[i], xy[i + spacing]
		v1, v2 = p - e1, e2 - p
		l1, l2, l3 = np.linalg.norm(v1), np.linalg.norm(v2), np.linalg.norm(e2 - e1)
		denom = l1 * l2 * l3
		if denom == 0:
			continue
		cross_z = v1[0] * v2[1] - v1[1] * v2[0]
		k = (abs(cross_z) / denom) * np.sign(cross_z)
		curv.append(k)

	if not curv:
		return np.nan, np.nan

	curv = np.array(curv)
	accumulated_curv = np.nanmean(np.abs(curv))
	net_curv = np.abs(np.nanmean(curv))
	return accumulated_curv, net_curv

def render_filaments(df, filename, viewer):
	onefile = df[df["Filename"] == filename]
	imp = BioImage(onefile["Filename"][0], reader=bioio_tifffile.Reader)
	image_data = imp.data[0, 0, :, :, :]
	scale = (imp.physical_pixel_sizes.Z, imp.physical_pixel_sizes.Y, imp.physical_pixel_sizes.X)
	viewer.add_image(image_data, scale=scale)
	coordlist = onefile["Coordinates (um)"].tolist()
	label_img = np.zeros(image_data.shape, dtype=np.uint16)
	coordlist = [np.round(coord[:, ::-1]/scale).astype(int) for coord in coordlist]
	for label_id, coord in enumerate(coordlist):
		for index in range(coord.shape[0]):
			label_img[(coord[index, 0], coord[index, 1], coord[index, 2])] = label_id + 1
	viewer.add_labels(label_img, scale=scale)

In [4]:
# Input "Per_Filament_Coordinates.csv" filepath here
folder_path = r"D:\Data\Durham\Output - Copy"
filename = "Per_Filament_Coordinates.csv"

In [ ]:
# Reads in data
datafile = os.path.join(folder_path, filename)
data = pd.read_csv(datafile)
# Parses coordinate strings into numpy arrays
data["Coordinates (um)"] = data["Coordinates (um)"].apply(parse_coordinates)
data["Verticies"] = data["Verticies"].apply(parse_coordinates, tuple_format=True)
# Parses vertex locations into lists
data["Vertex Locations"] = data["Vertex Locations"].apply(parse_list)
# Getting number of pixels in filament
data["Num. Pixels"] = data["Coordinates (um)"].apply(lambda coords: len(coords))
# Filtering by minimum pixels
data = data[data["Num. Pixels"] >= min_pixels]

# Getting length of filament in 2D
data["Length 2D (um)"] = data["Coordinates (um)"].apply(get_distance, dims=2)
# Filtering by minimum lengths in 2D
data = data[data["Length 2D (um)"] >= min_length_2D]
# Getting length of filament in 3D
data["Length 3D (um)"] = data["Coordinates (um)"].apply(get_distance, dims=3)
# Filtering by minimum lengths in 3D
data = data[data["Length 3D (um)"] >= min_length_3D]

# Gets angles by getting angle between points spaced by 'spacer' value and averaging them
data["Average Angle (radians)"] = data["Coordinates (um)"].apply(get_average_radians, spacer=5)
data["Average Angle (degrees)"] = np.degrees(data["Average Angle (radians)"])
# Gets angle between the first and last point of the filament
data["End to End Angle (radians)"] = data["Coordinates (um)"].apply(lambda coords: get_radians_2D(coords[0], coords[-1]))
data["End to End Angle (degrees)"] = np.degrees(data["End to End Angle (radians)"])

# Gets branch angles by finding filaments that share verticies and comparing their average angles
results = []
for (fname, chan), subset in data.groupby(["Filename", "Channel"]):
	results.append(get_branch_angles(subset, "Verticies", "Average Angle (degrees)", "Branch Angles (degrees)"))
data = pd.concat(results)

#  Gets the mean distance from a straight line between the endpoints of the filament
data["Deviation (um)"] = data["Coordinates (um)"].apply(get_residuals)

# Gets curvature values
data[["Accumulated Curvature", "Net Curvature"]] = data["Coordinates (um)"].apply(lambda coords: get_curvature(coords, spacing=1)).apply(pd.Series)

databackup = data.copy()

In [109]:
data = databackup.copy()

This next cell saves the outputs

In [27]:
data.to_pickle(os.path.join(folder_path, "Per_Filament_Coordinates_Analyzed.pkl"))

This next cell can load in previously saved outputs

In [5]:
data = pd.read_pickle(os.path.join(folder_path, "Per_Filament_Coordinates_Analyzed.pkl"))
databackup = data.copy()

This next cell opens up napari

In [6]:
Viewer = napari.Viewer()

This next cell allows you to view a labelled image

In [7]:
render_filaments(data, data["Filename"][0], Viewer)